<a href="https://colab.research.google.com/github/aharrisonau/Task-3.2D/blob/main/Task_3_2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#SIT796 Reinforcement Learning
##Distinction Task 3.2: Exact policy iteration implementation for MDPs

This task follows up 1.2C, which was the definition of a truck trailer reversing problem.

In this task, the aim is to set up a policy value mapping for truck reversal.  To achieve this, the state space has been mapped to 4 variables;
- pivot X [0, 2, 4,... 30] metres
- pivot Y [-10, -8, -6, .... 35] metres
- prime mover angle [0, 2pi/10, 4pi/10, ... 18pi/10] radians
- trailer angle [0, 2pi/10, 4pi/10, ... 18pi/10] radians

This gives 16 x 21 x 10 x 10 = 33600 individual co-ordinates, which is a balance between space state size and resolution

Reward of 1 is achieved with;
- pivot X,Y = (10,0)
- trailer angle = 0
- pm angle = [16pi/10, 18pi/10, 0, 2pi/10, 4pi/10]  NOTE: other PM angles represent a jack-knifed trailer and should not occur


In [ ]:
# preliminaries
import numpy as np
import pandas as pd
import math
pi = math.pi
import cmath


In [ ]:
# the calculations will be done in numpy for speed
# first define the State Space co-ordinated for the array
SSpivotX = np.linspace(0,30,16)
SSpivotY = np.linspace(-10,30,21)
SSpmAngle = np.linspace(-10*pi/10,8*pi/10,10)
SStrlAngle = np.linspace(-10*pi/10,8*pi/10,10)

# then set the reward space and terminal nodes as all zeros
Rewards = np.zeros((len(SSpivotX),len(SSpivotY),len(SSpmAngle),len(SStrlAngle)))
TerminalStates = np.zeros((len(SSpivotX),len(SSpivotY),len(SSpmAngle),len(SStrlAngle)))

# and lastly set the terminals for backed in (no next state)
TerminalStates[(5,5,range(-2,3),0)] = 1
Rewards[(5,5,range(-2,3),0)] = 1

In [ ]:
# define a function that duplicates the step function of the gym environment

def _2D_Vector_Rotation(_2D_Vector,rotationAngle): # helper function to rotate vectors
    import numpy as np
    rotationMatrix = np.array([[np.cos(rotationAngle),-np.sin(rotationAngle)],
                           [np.sin(rotationAngle),np.cos(rotationAngle)]])
    return np.matmul(rotationMatrix,_2D_Vector)

def nextStateFN(state, action): # function is a modified version of the environment step function
    pmX,pmY,pivX,pivY,trlX,trlY = state
    move,steer = action
    TruckDefinition = np.array( [5.0, 2.5, 10.0, 2.5])
    """
    in each step, 
    - the prime mover will go back 2m (at the pivot) - modified to match state space definition
    - the prime mover will rotate (front relative to pivot)
    - the trailer back will rotate due to its relative angle
    """

    pmVect=np.array([pmX-pivX,pmY-pivY]) # prime mover vector
    trlVect=np.array([pivX-trlX,pivY-trlY]) # trailer vector
    pivotMovement=np.linalg.norm(pmVect)*move*2.0 # amount the pivot moves
    pmAngleDelta=math.asin((math.tan(math.pi/4*steer)*move*2.0/TruckDefinition[0])) # the change in PM angle due to wheel steering
    pm_trlSin = np.cross(trlVect,pmVect,)/np.linalg.norm(pmVect)/np.linalg.norm(trlVect) #sine of the trailer angle relative to the PM
    pm_trlAngle=math.asin(np.clip(pm_trlSin,-1,1)) #relative trailer angle.  Clip eliminate rounding errors giving arguments >1
    trlAngleDelta=math.asin((math.sin(pm_trlAngle)*move*2.0/TruckDefinition[2])) # the change in trailer angle
    
    # move the pivot point
    pmUnitVector = pmVect/np.linalg.norm(pmVect)
    [pivX,pivY] = [pivX,pivY] + move * 2.0 *pmUnitVector
    
    # rotate the prime mover and adjust the front location
    pmVect = _2D_Vector_Rotation(pmVect,pmAngleDelta)
    [pmX,pmY] = [pivX,pivY] + pmVect
    
    # rotate the trailer and adjust its back location
    trlVect = _2D_Vector_Rotation(trlVect,trlAngleDelta)
    [trlX,trlY] = [pivX,pivY] - trlVect
    nextState = pmX,pmY,pivX,pivY,trlX,trlY

    done = False

    reward = 0.0

    return nextState

Now we step through every state in the state space and use the nextState function to get the following state for a given action.

The actions that we will use are;
move = constant -1 (which is moving 2m back)
steer = [-1, -0.2, 0, 0.2, 1] (corresponding from full lock right to full lock left)

We will check if the next state is a terminal state (out of bounds) or has a reward as well and record those results

These can then be used for value iteration for the episodic event chain

In [ ]:
# set up some bins to pull state calcuations into the nearest state space points
binsPivotX = SSpivotX+1
binsPivotY = SSpivotY+1
binsPmAngle = SSpmAngle+pi/10
binsTrlAngle = SStrlAngle+pi/10

In [ ]:
# set the allowable actions
move = -1.0 #only going backwards
steering = np.array([-1, -0.2, 0, 0.2, 1])

In [ ]:
# use a nested dictionary structure to store the results
# levels will be
# 1 - PivotX bins
# 2 - PivotY bins
# 3 - PmAngle bins
# 4 - TrlAngle bins
# 5 - action index

nextStateDict = {}

for pivX in SSpivotX:
  print(pivX,",",end=" ") # heartbeat to follow progress
  nextStateDict[pivX] = {}
  for pivY in SSpivotY:
    nextStateDict[pivX][pivY] = {}
    for pmAngle in SSpmAngle:
      nextStateDict[pivX][pivY][pmAngle] = {}
      for trlAngle in SStrlAngle:
        nextStateDict[pivX][pivY][pmAngle][trlAngle] = {}
        nextStateDict[pivX][pivY][pmAngle][trlAngle]["value"]=0
        # note that the value of a state is stored against the state
        for steer in steering:
          # this is now one of the 5 actions applied at a particular state
          nextStateDict[pivX][pivY][pmAngle][trlAngle][steer] = {}
          # the gym environment gives a state in 3 pairs of x-y coordinates
          # in order to reduce the state space, it has one pair of x-y and two angles
          # so we need to conver the state space to the 3 pairs
          pmVec = _2D_Vector_Rotation([5.0,0.0],pmAngle)
          [pmX,pmY] = [pivX,pivY] + pmVec
          trlVec = _2D_Vector_Rotation([10.0,0.0],trlAngle)
          [trlX,trlY] = [pivX,pivY] - trlVec
          # get the next state using the same method as the gym environment
          state = pmX, pmY, pivX, pivY, trlX, trlY
          nextState = nextStateFN(state,(-1.0,steer))
          # get the coordinates of the next state
          next_pmX,next_pmY,next_pivX,next_pivY,next_trlX,next_trlY = nextState
          # check if the state has gone out of bounds.  If so, zero it out
          if  not 5.0 <= next_pivX <= 25:
            nextState = 0
          if not -5.0 <= next_pivY <= 30:
            nextState = 0
          # convert the 3 pairs back to the one pair and two angles
          # and then bin them into their correct index
          # NOTE:  Better to put everything into the dictionary in the future
          _,nextPmAngle = cmath.polar(complex(next_pmX-next_pivX,next_pmY-next_pivY))
          nextSSpmAngle = np.searchsorted(binsPmAngle,nextPmAngle) % 10
          _,nextTrlAngle = cmath.polar(complex(next_pivX-next_trlX,next_pivY-next_trlY))
          nextSStrlAngle = np.searchsorted(binsTrlAngle,nextTrlAngle) % 10
          nextSSpivotX = np.searchsorted(binsPivotX,next_pivX)
          nextSSpivotY = np.searchsorted(binsPivotY,next_pivY)
          # get the indices of the next state space
          nextSS = (nextSSpivotX, nextSSpivotY,nextSSpmAngle,nextSStrlAngle)
          # if the next state is zero, it is out of bounds
          # set the next state for the State,Action combination
          # and the reward for it is zero
          if nextState == 0:
            nextStateDict[pivX][pivY][pmAngle][trlAngle][steer]["nextSS"] = 0
            nextStateDict[pivX][pivY][pmAngle][trlAngle][steer]["reward"] = 0
          else:
            # check for the reward on the next state
            reward = Rewards[nextSS]
            # assign that reward for that action at that state
            nextStateDict[pivX][pivY][pmAngle][trlAngle][steer]["reward"] = reward
            # if the reward is greater than the current value of the state, update it
            nextStateDict[pivX][pivY][pmAngle][trlAngle]["value"] = max(reward, 
                        nextStateDict[pivX][pivY][pmAngle][trlAngle]["value"])   
            # record what the next state for that action on this state is                                             
            nextStateDict[pivX][pivY][pmAngle][trlAngle][steer]["nextSS"] = nextSS



          


0.0 , 2.0 , 4.0 , 6.0 , 8.0 , 10.0 , 12.0 , 14.0 , 16.0 , 18.0 , 20.0 , 22.0 , 24.0 , 26.0 , 28.0 , 30.0 , 

Now we can use a state value iteration process, as well as a policy iteration process to determine a map of optimum values and actions.

Essentially, we pass through the whole state space, checking all the actions at each point and assessing the value at the next space that action reaches.

We record the best action to take, plus copy a slightly discounted value from that next state to the current state.

This pass is repeated until sufficient convergence is achieved.

In [ ]:
# loop through the state space and update the rewards
stillConverging = True
count = 0
gamma = 0.99  # discount factor
changeLimit = 0.001

while stillConverging:
  
  change = 0.0

  for pivX in SSpivotX:
    print(pivX,",",end=" ") # heartbeat to follow progress
    for pivY in SSpivotY:
      for pmAngle in SSpmAngle:
        for trlAngle in SStrlAngle:
          # reset variables to track the best value and action for this state
          bestValue = 0
          bestAction = 0
          # loop through the actions
          for steer in steering:
            nextSS = nextStateDict[pivX][pivY][pmAngle][trlAngle][steer]["nextSS"]
            if nextSS != 0: # if it is zero, that action was out-of-bounds
              nextSSvalue = nextStateDict[SSpivotX[nextSS[0]]] \
                                          [SSpivotY[nextSS[1]]] \
                                          [SSpmAngle[nextSS[2]]] \
                                          [SStrlAngle[nextSS[2]]] \
                                          ["value"]
              if nextSSvalue > bestValue: # if this action was better
                bestValue = nextSSvalue   # update the best value
                bestAction = steer        # update the best action
          
          oldValue = nextStateDict[pivX][pivY][pmAngle][trlAngle]["value"]
          diff = abs(oldValue - bestValue)
          change = max(diff,change) # check if this value change is the biggest
          nextStateDict[pivX][pivY][pmAngle][trlAngle]["value"]=bestValue*gamma
          nextStateDict[pivX][pivY][pmAngle][trlAngle]["bestAction"]=bestAction

  print(count,change)
  count += 1 # update the loop count
  stillConverging = change > changeLimit # stop the loop once the largest change is small

0.0 , 2.0 , 4.0 , 6.0 , 8.0 , 10.0 , 12.0 , 14.0 , 16.0 , 18.0 , 20.0 , 22.0 , 24.0 , 26.0 , 28.0 , 30.0 , 0 1.0
0.0 , 2.0 , 4.0 , 6.0 , 8.0 , 10.0 , 12.0 , 14.0 , 16.0 , 18.0 , 20.0 , 22.0 , 24.0 , 26.0 , 28.0 , 30.0 , 1 0.99
0.0 , 2.0 , 4.0 , 6.0 , 8.0 , 10.0 , 12.0 , 14.0 , 16.0 , 18.0 , 20.0 , 22.0 , 24.0 , 26.0 , 28.0 , 30.0 , 2 0.9801
0.0 , 2.0 , 4.0 , 6.0 , 8.0 , 10.0 , 12.0 , 14.0 , 16.0 , 18.0 , 20.0 , 22.0 , 24.0 , 26.0 , 28.0 , 30.0 , 3 0.9702989999999999
0.0 , 2.0 , 4.0 , 6.0 , 8.0 , 10.0 , 12.0 , 14.0 , 16.0 , 18.0 , 20.0 , 22.0 , 24.0 , 26.0 , 28.0 , 30.0 , 4 0.9135172474836407
0.0 , 2.0 , 4.0 , 6.0 , 8.0 , 10.0 , 12.0 , 14.0 , 16.0 , 18.0 , 20.0 , 22.0 , 24.0 , 26.0 , 28.0 , 30.0 , 5 0.9043820750088043
0.0 , 2.0 , 4.0 , 6.0 , 8.0 , 10.0 , 12.0 , 14.0 , 16.0 , 18.0 , 20.0 , 22.0 , 24.0 , 26.0 , 28.0 , 30.0 , 6 0.8953382542587163
0.0 , 2.0 , 4.0 , 6.0 , 8.0 , 10.0 , 12.0 , 14.0 , 16.0 , 18.0 , 20.0 , 22.0 , 24.0 , 26.0 , 28.0 , 30.0 , 7 0.8863848717161291
0.0 , 2.0 , 4.0 ,

Once the value and best action map is complete, it can be examined.  As it is four dimensional, it is best viewed as a map of the x-y co-ords for a fixed prime mover and trailer angle set.  In this case, the map is for angles of 4*pi/10

In [ ]:
map = pd.DataFrame(index=SSpivotY)

for pivX in SSpivotX:
  col = []
  for pivY in SSpivotY:
    pmAngle = SSpmAngle[7]
    trlAngle = SStrlAngle[7]
    col.append(nextStateDict[pivX][pivY][pmAngle][trlAngle]["value"]) 

  map[pivX] = col

pd.options.display.float_format = '{:.2f}'.format
map


,0.00,2.00,4.00,6.00,8.00,10.00,12.00,14.00,16.00,18.00,20.00,22.00,24.00,26.00,28.00,30.00
-10.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
-8.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
-6.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
-4.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
-2.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.09,0.09,0.09,0.09,0.09,0.09,0.09,0.00,0.00,0.00
2.00,0.00,0.00,0.00,0.09,0.09,0.09,0.09,0.09,0.09,0.09,0.09,0.09,0.09,0.00,0.00,0.00
4.00,0.00,0.00,0.00,0.09,0.09,0.09,0.09,0.09,0.09,0.09,0.09,0.09,0.09,0.00,0.00,0.00
6.00,0.00,0.00,0.00,0.09,0.09,0.09,0.09,0.09,0.09,0.09,0.09,0.09,0.09,0.00,0.00,0.00
8.00,0.00,0.00,0.00,0.09,0.09,0.09,0.09,0.09,0.09,0.09,0.09,0.09,0.09,0.00,0.00,0.00


From any state point that has a value, this value has been propogated back through a chain of actions that leads to the goal.  This can be followed forwards as a the optimum path to achieve the goal

In [38]:
!wget -nc https://raw.githubusercontent.com/brpy/colab-pdf/master/colab_pdf.py
from colab_pdf import colab_pdf
colab_pdf('Task 3.2D.ipynb')

File ‘colab_pdf.py’ already there; not retrieving.





Extracting templates from packages: 100%
[NbConvertApp] Converting notebook /content/drive/My Drive/Colab Notebooks/Task 3.2D.ipynb to pdf
[NbConvertApp] Writing 71031 bytes to ./notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: [u'xelatex', u'./notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: [u'bibtex', u'./notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 53817 bytes to /content/drive/My Drive/Task 3.2D.pdf


'File Download Unsuccessful. Saved in Google Drive'